In [1]:
import os
import pathlib

import pandas as pd
import pypsa
import numpy as np
import plotly.express as px
import xarray as xr


In [2]:

def get_network_map(pypsa_earth_path):
    RESULTS_DIR=os.path.join(pypsa_earth_path,"results")
    networks = {}
    for dir in os.listdir(RESULTS_DIR):
        entry = pathlib.Path(RESULTS_DIR, dir)
        if not entry.is_dir():
             continue



        for dir_child in os.listdir(pathlib.Path(entry, "networks")):
            if not dir_child.endswith(".nc"):
                continue
            networks[(dir, dir_child)] = pypsa.Network(pathlib.Path(entry, "networks", dir_child))
    print(networks)
    return networks



In [3]:
network_map=get_network_map("../pypsa-earth")

INFO:pypsa.io:Imported network elec_s_10_ec_lcopt_Co2L-4H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network elec_s_10_ec_lcopt_Co2L-4H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network elec_s_10_ec_lcopt_Co2L-4H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


{('MA', 'elec_s_10_ec_lcopt_Co2L-4H.nc'): PyPSA Network
Components:
 - Bus: 30
 - Carrier: 18
 - Generator: 42
 - GlobalConstraint: 1
 - Line: 13
 - Link: 40
 - Load: 10
 - StorageUnit: 6
 - Store: 20
Snapshots: 36, ('NG', 'elec_s_10_ec_lcopt_Co2L-4H.nc'): PyPSA Network
Components:
 - Bus: 33
 - Carrier: 18
 - Generator: 42
 - GlobalConstraint: 1
 - Line: 11
 - Link: 44
 - Load: 11
 - StorageUnit: 2
 - Store: 22
Snapshots: 36, ('NG_no_oil', 'elec_s_10_ec_lcopt_Co2L-4H.nc'): PyPSA Network
Components:
 - Bus: 33
 - Carrier: 16
 - Generator: 40
 - GlobalConstraint: 1
 - Line: 11
 - Link: 44
 - Load: 11
 - StorageUnit: 2
 - Store: 22
Snapshots: 36}


In [4]:
for _, network in network_map.items():
    print(network.statistics()["Capacity Factor"].loc["Generator"][["Solar", "load"]].values)

[1.55685243e-01 3.69973086e-19]
[1.74397064e-01 9.75284611e-19]
[1.74400172e-01 1.35849712e-12]


In [5]:
da=xr.open_dataset("../data/resources/NG/renewable_profiles/profile_solar.nc")

da.values

<bound method Mapping.values of <xarray.Dataset>
Dimensions:           (time: 144, bus: 157, y: 34, x: 42)
Coordinates:
  * time              (time) datetime64[ns] 2013-03-01 ... 2013-03-06T23:00:00
  * bus               (bus) object '0' '2' '4' '5' ... '195' '196' '197' '198'
  * y                 (y) float64 4.2 4.5 4.8 5.1 5.4 ... 13.2 13.5 13.8 14.1
  * x                 (x) float64 2.4 2.7 3.0 3.3 3.6 ... 13.8 14.1 14.4 14.7
Data variables:
    profile           (time, bus) float64 ...
    weight            (bus) float64 ...
    p_nom_max         (bus) float64 ...
    potential         (y, x) float64 ...
    average_distance  (bus) float64 ...>

In [6]:
da.keys()

KeysView(<xarray.Dataset>
Dimensions:           (time: 144, bus: 157, y: 34, x: 42)
Coordinates:
  * time              (time) datetime64[ns] 2013-03-01 ... 2013-03-06T23:00:00
  * bus               (bus) object '0' '2' '4' '5' ... '195' '196' '197' '198'
  * y                 (y) float64 4.2 4.5 4.8 5.1 5.4 ... 13.2 13.5 13.8 14.1
  * x                 (x) float64 2.4 2.7 3.0 3.3 3.6 ... 13.8 14.1 14.4 14.7
Data variables:
    profile           (time, bus) float64 ...
    weight            (bus) float64 ...
    p_nom_max         (bus) float64 ...
    potential         (y, x) float64 ...
    average_distance  (bus) float64 ...)

In [7]:
da.items()

ItemsView(<xarray.Dataset>
Dimensions:           (time: 144, bus: 157, y: 34, x: 42)
Coordinates:
  * time              (time) datetime64[ns] 2013-03-01 ... 2013-03-06T23:00:00
  * bus               (bus) object '0' '2' '4' '5' ... '195' '196' '197' '198'
  * y                 (y) float64 4.2 4.5 4.8 5.1 5.4 ... 13.2 13.5 13.8 14.1
  * x                 (x) float64 2.4 2.7 3.0 3.3 3.6 ... 13.8 14.1 14.4 14.7
Data variables:
    profile           (time, bus) float64 ...
    weight            (bus) float64 ...
    p_nom_max         (bus) float64 ...
    potential         (y, x) float64 ...
    average_distance  (bus) float64 ...)

In [8]:
da.to_dataframe()


profile       weight     p_nom_max   
time                bus y    x                                          
2013-03-01 00:00:00 0   4.2  2.4       0.0  1986.826712  11132.924304  \
                             2.7       0.0  1986.826712  11132.924304   
                             3.0       0.0  1986.826712  11132.924304   
                             3.3       0.0  1986.826712  11132.924304   
                             3.6       0.0  1986.826712  11132.924304   
...                                    ...          ...           ...   
2013-03-06 23:00:00 198 14.1 13.5      0.0   949.203421   5930.869005   
                             13.8      0.0   949.203421   5930.869005   
                             14.1      0.0   949.203421   5930.869005   
                             14.4      0.0   949.203421   5930.869005   
                             14.7      0.0   949.203421   5930.869005   

                                   potential  average_distance  
time                bus y    x                                  
2013-03-01 00:00:00 0   4.2  2.4         0.0         37.391130  
                             2.7         0.0         37.391130  
                             3.0         0.0         37.391130  
                             3.3         0.0         37.391130  
                             3.6         0.0         37.391130  
...                                      ...               ...  
2013-03-06 23:00:00 198 14.1 13.5        0.0         28.105624  
                             13.8        0.0         28.105624  
                             14.1        0.0         28.105624  
                             14.4        0.0         28.105624  
                             14.7        0.0         28.105624  

[32284224 rows x 5 columns]

In [30]:
for k,v in network_map.keys():
    print(v)
    print(k)
    print(network_map.get((k,v)))

# print(network_map.keys())
# network_map.values()
print(network_map.keys())

elec_s_10_ec_lcopt_Co2L-4H.nc
MA
PyPSA Network
Components:
 - Bus: 30
 - Carrier: 18
 - Generator: 42
 - GlobalConstraint: 1
 - Line: 13
 - Link: 40
 - Load: 10
 - StorageUnit: 6
 - Store: 20
Snapshots: 36
elec_s_10_ec_lcopt_Co2L-4H.nc
NG
PyPSA Network
Components:
 - Bus: 33
 - Carrier: 18
 - Generator: 42
 - GlobalConstraint: 1
 - Line: 11
 - Link: 44
 - Load: 11
 - StorageUnit: 2
 - Store: 22
Snapshots: 36
elec_s_10_ec_lcopt_Co2L-4H.nc
NG_no_oil
PyPSA Network
Components:
 - Bus: 33
 - Carrier: 16
 - Generator: 40
 - GlobalConstraint: 1
 - Line: 11
 - Link: 44
 - Load: 11
 - StorageUnit: 2
 - Store: 22
Snapshots: 36
dict_keys([('MA', 'elec_s_10_ec_lcopt_Co2L-4H.nc'), ('NG', 'elec_s_10_ec_lcopt_Co2L-4H.nc'), ('NG_no_oil', 'elec_s_10_ec_lcopt_Co2L-4H.nc')])
